This is a tutorial to use the flip package: https://github.com/corentinravoux/flip \
It is self-contained and can be used in google collab or on your environement \
All the data used are subsampled version of a simulation. \
The data size is small for the tutorial, do not use it for science case. \

In [5]:
!pip uninstall -y jax

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Found existing installation: jax 0.4.30
Uninstalling jax-0.4.30:
  Successfully uninstalled jax-0.4.30


In [1]:
!pip install jax

Defaulting to user installation because normal site-packages is not writeable


In [2]:
%%capture
!pip install ../


In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from flip import fitter, plot_utils, utils, vectors
from flip.covariance import covariance, contraction
from pkg_resources import resource_filename
flip_base = resource_filename("flip", ".")
data_path = os.path.join(flip_base, "data")
plt.style.use(os.path.join(data_path,"style.mplstyle"))

/home/ravoux/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
[ 000000.00 ]: 07-26 16:58  root            INFO     No pypower module detected, gridding with this method is unavailable
[ 000000.00 ]: 07-26 16:58  root            WARNING  Install CLASS https://github.com/lesgourg/class_public to use class.py module


In [4]:
use_jit = True
use_gradient = True

In [5]:
grid1 = pd.read_parquet(os.path.join(data_path,"density_data.parquet"))
grid2 = grid1.copy()
grid2["x"] = grid2["x"] + 200
grid3 = grid1.copy()
grid3["x"] = grid1["x"] - 200
grid = pd.concat([grid1])

grid_window = pd.read_parquet(os.path.join(data_path,"grid_window_m.parquet"))

coordinates_density = np.array([grid["ra"], grid["dec"], grid["rcom"]])
data_density = {'density': np.array(grid["density"]), 
                'density_error': np.array(grid["density_err"])}


ktt, ptt = np.loadtxt(os.path.join(data_path,"power_spectrum_tt.txt"))
kmt, pmt = np.loadtxt(os.path.join(data_path,"power_spectrum_mt.txt"))
kmm, pmm = np.loadtxt(os.path.join(data_path,"power_spectrum_mm.txt"))

sigmag_fiducial = 3.0

power_spectrum_dict_bias = {"gg": [[kmm, pmm * np.array(grid_window["window_mm"])**2]]}
power_spectrum_dict = {"gg": [[kmm, pmm * np.array(grid_window["window_mm"])**2],
                              [kmt, pmt * np.array(grid_window["window_mt"])],
                              [ktt, ptt]]}

size_batch=10_000
number_worker=8

covariance_fit = covariance.CovMatrix.init_from_flip(
    "adamsblake20",
    "density",
    power_spectrum_dict,
    coordinates_density=coordinates_density,
    size_batch=size_batch,
    number_worker=number_worker,
    additional_parameters_values=(sigmag_fiducial,),
)


likelihood_type="multivariate_gaussian"
likelihood_properties = {"inversion_method": "cholesky",
                        "use_jit":use_jit,
                        "use_gradient":use_gradient}

parameter_dict = {"bs8": {"value": 1.0,
                          "limit_low" : 0.0,
                          "fixed" : False,
                         },
                  "beta_f": {"value": 0.4,
                          "limit_low" : 0.0,
                          "fixed" : False,
                         },
                 }




minuit_fitter = fitter.FitMinuit.init_from_covariance(
        covariance_fit,
        data_density,
        parameter_dict,
        likelihood_type=likelihood_type,
        likelihood_properties = likelihood_properties)

minuit_fitter.run()



[ 000004.02 ]: 07-26 16:58  jax._src.xla_bridge INFO     Unable to initialize backend 'cuda': 
[ 000004.02 ]: 07-26 16:58  jax._src.xla_bridge INFO     Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
[ 000004.03 ]: 07-26 16:58  jax._src.xla_bridge INFO     Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
[ 000004.03 ]: 07-26 16:58  jax._src.xla_bridge INFO     Unable to initialize backend 'cuda': 
[ 000004.03 ]: 07-26 16:58  jax._src.xla_bridge INFO     Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
[ 000004.03 ]: 07-26 16:58  jax._src.xla_bridge INFO     Unable to initialize backend 'cuda': 
[ 000004.03 ]: 07-26 16:58  jax._src.xla_bridge INFO     Unable to initialize backend 'cuda': 
[ 000004.03 ]: 07-26 16:58  jax._src.xla_bridge INFO     Unable to initialize backend 'rocm': modu

{'bs8': 0.8228999982398881, 'beta_f': 0.3086451044958614}

In [6]:
data_velocity = pd.read_parquet(os.path.join(data_path, "velocity_data.parquet"))

data_velocity_true = {"velocity": data_velocity["vpec"].values, 
                      "velocity_error": np.zeros(len(data_velocity["vpec"].values))}

coordinates_velocity = np.array([data_velocity["ra"], 
                                 data_velocity["dec"], 
                                 data_velocity["rcom_zobs"]])

ktt, ptt = np.loadtxt(os.path.join(data_path, "power_spectrum_tt.txt"))
kmt, pmt = np.loadtxt(os.path.join(data_path, "power_spectrum_mt.txt"))
kmm, pmm = np.loadtxt(os.path.join(data_path, "power_spectrum_mm.txt"))

sigmau_fiducial = 15

power_spectrum_dict = {"vv": [[ktt, ptt * utils.Du(ktt, sigmau_fiducial) ** 2]]}

size_batch=10_000
number_worker=16


covariance_fit = covariance.CovMatrix.init_from_flip(
    "carreres23",
    "velocity",
    power_spectrum_dict,
    coordinates_velocity=coordinates_velocity,
    size_batch=size_batch,
    number_worker=number_worker,
)

likelihood_type="multivariate_gaussian"
likelihood_properties = {"inversion_method": "cholesky",
                         "velocity_type": "direct",
                         "use_jit": use_jit,
                         "use_gradient": use_gradient}


parameter_dict = {"fs8": {"value": 0.4,
                          "limit_low" : 0.0,
                          "fixed" : False,
                         },
                  "sigv": {"value": 200,
                          "limit_low" : 0.0,
                          "fixed" : False,
                         },
                 }



minuit_fitter = fitter.FitMinuit.init_from_covariance(
        covariance_fit,
        data_velocity_true,
        parameter_dict,
        likelihood_type=likelihood_type,
        likelihood_properties = likelihood_properties
)


minuit_fitter.run()

# parameter_values = [
#             parameter_dict[parameters]["value"] for parameters in parameter_dict
#         ]
# minuit_fitter.likelihood(parameter_values)


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
[ 000099.26 ]: 07-26 15:02  root            INFO     Covariance matrix generated from flip with carreres23 model in 5.70e-01 seconds
[ 000102.90 ]: 07-26 15:02  root            INFO     ┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 1.374e+04                  │         Nfcn = 35, Ngrad = 2         │
│ EDM = 3.49e-05 (Goal: 0.0002)    │            time = 3.6 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────

{'fs8': 0.3527380146365558, 'sigv': 199.53910751729765}

In [7]:
data_velocity = pd.read_parquet(os.path.join(data_path, "velocity_data.parquet"))

data_velocity_true = {"velocity": data_velocity["vpec"].values, 
                      "velocity_error": np.zeros(len(data_velocity["vpec"].values))}

coordinates_velocity = np.array([data_velocity["ra"], 
                                 data_velocity["dec"], 
                                 data_velocity["rcom_zobs"]])

ktt, ptt = np.loadtxt(os.path.join(data_path, "power_spectrum_tt.txt"))
kmt, pmt = np.loadtxt(os.path.join(data_path, "power_spectrum_mt.txt"))
kmm, pmm = np.loadtxt(os.path.join(data_path, "power_spectrum_mm.txt"))

sigmau_fiducial = 15

power_spectrum_dict = {"vv": [[ktt, ptt * utils.Du(ktt, sigmau_fiducial) ** 2]]}

size_batch=10_000
number_worker=16

sigmau_list = np.linspace(10.0, 20.0,10)
covariance_list = []
size_batch=10_000
number_worker=16


for sigu in sigmau_list:
    
    power_spectrum_dict = {"vv": [[ktt, ptt* utils.Du(ktt, sigu)**2]]}

    covariance_list.append(covariance.CovMatrix.init_from_flip(
                            "carreres23",
                            "velocity",
                            power_spectrum_dict,
                            coordinates_velocity=coordinates_velocity,
                            size_batch=size_batch,
                            number_worker=number_worker,
    ))
    
likelihood_type="multivariate_gaussian_interp1d"
likelihood_properties = {"inversion_method": "cholesky",
                         "velocity_type": "direct",
                         "use_jit": use_jit,
                         "use_gradient": use_gradient,}

parameter_dict = {"fs8": {"value": 0.4,
                          "limit_low" : 0.0,
                          "fixed" : False,
                         },
                  "sigv": {"value": 200,
                          "limit_low" : 0.0,
                          "fixed" : False,
                         },
                  "sigu": {"value": 15.0,
                          "limit_low" : 13.0,
                          "limit_up" : 17.0,
                          "fixed" : False,
                         },
                 }


minuit_fitter_interp = fitter.FitMinuit.init_from_covariance(
        covariance_list,
        data_velocity_true,
        parameter_dict,
        likelihood_type=likelihood_type,
        likelihood_properties = likelihood_properties,
        interpolation_value_name = "sigu",
        interpolation_value_range = sigmau_list,
)


minuit_fitter_interp.run()


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
[ 000108.42 ]: 07-26 15:02  root            INFO     Covariance matrix generated from flip with carreres23 model in 5.67e-01 seconds
[ 000109.02 ]: 07-26 15:02  root            INFO     Covariance matrix generated from flip with carreres23 model in 5.99e-01 seconds
[ 000109.61 ]: 07-26 15:02  root            INFO     Covariance matrix generated from flip with carreres23 model in 5.86e-01 seconds
[ 000110.21 ]: 07-26 15:02  root            INFO     Covariance matrix generated from flip with carreres23 model in 6.00e-01 seconds
[ 000110.81 ]: 07-26 15:02  root            INFO     Covariance matrix generated from flip with carreres23 model in 6.01e-01 seconds
[ 000111.41 ]: 07-26 15:02  root            INFO     Covariance matrix generated from flip with carrere

{'fs8': 0.36614482471524257,
 'sigv': 200.8121744175942,
 'sigu': 16.99972091553989}

In [8]:
test_parameters = {"alpha":0.14,
                   "beta": 3.1,
                   "M_0": -19.133,
                   "sigma_M": 0.12}

velocity, velocity_error = vectors.get_velocity_from_salt_fit(
    data_velocity,
    test_parameters,
    "full",
)

mask = np.abs(velocity) < 3_000
mask &= velocity_error < 3_000

coordinates_velocity_cleaned = np.transpose(np.transpose(coordinates_velocity)[mask])
data_velocity_cleaned = {}
for key in list(data_velocity.keys()):
    data_velocity_cleaned[key] = np.array(data_velocity[key])[mask]
    
size_batch=10_000
number_worker=16


covariance_fit = covariance.CovMatrix.init_from_flip(
    "carreres23",
    "velocity",
    power_spectrum_dict,
    coordinates_velocity=coordinates_velocity_cleaned,
    size_batch=size_batch,
    number_worker=number_worker,
)


likelihood_type="multivariate_gaussian"
likelihood_properties = {"inversion_method": "cholesky",
                         "velocity_type": "saltfit",
                         "velocity_estimator": "full",
                         "use_jit": use_jit,
                         "use_gradient": use_gradient,
                        }


parameter_dict = {"fs8": {"value": 0.4,
                          "limit_low" : 0.0,
                          "limit_up" : 1.0,
                          "fixed" : False,
                         },
                  "sigv": {"value": 200,
                          "limit_low" : 0.0,
                          "limit_up" : 1000,
                          "fixed" : False,
                         },
                  "alpha": {"value": 0.1,
                          "limit_low" : 0.0,
                          "limit_up" : 0.25,
                          "fixed" : False,
                         },
                  "beta": {"value": 3.0,
                          "limit_low" : 2.5,
                          "limit_up" : 4.5,
                          "fixed" : False,
                         },
                  "M_0": {"value": -19,
                          "limit_low" : -21,
                          "limit_up" : -18,
                          "fixed" : False,
                         },
                  "sigma_M": {"value": 0.1,
                          "limit_low" : 0.0,
                          "limit_up" : 100.0,
                          "fixed" : False,
                         }}

minuit_fitter_velest = fitter.FitMinuit.init_from_covariance(
        covariance_fit,
        data_velocity_cleaned,
        parameter_dict,
        likelihood_type=likelihood_type,
        likelihood_properties = likelihood_properties
)


minuit_fitter_velest.run()


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
[ 000127.40 ]: 07-26 15:02  root            INFO     Covariance matrix generated from flip with carreres23 model in 6.18e-01 seconds
/home/ravoux/.local/lib/python3.10/site-packages/iminuit/util.py:172: IMinuitWarning: Assigned errors must be positive. Non-positive values are replaced by a heuristic.
  warnings.warn(
[ 000136.93 ]: 07-26 15:02  root            INFO     ┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 1.415e+04                  │        Nfcn = 139, Ngrad = 8         │
│ EDM = 1.38e-05 (Goal: 0.0002)    │            time = 9.5 sec            │
├───────────────

{'fs8': 0.47381309968970364,
 'sigv': 593.1049535697847,
 'alpha': 0.13285894991736366,
 'beta': 3.099666399471431,
 'M_0': -19.067819495828125,
 'sigma_M': 0.0863770652902524}